## National Technical University of Athens
### School of Electrical & Computer Engineering
### Course: **Computational Cryptography**
##### *9th Semester, 2021-2022*

<br>

###### Full Name: Christos Tsoufis
###### A.M.: 031 17 176

<br>

#### Complimentary Code for Homework #4


### Installation of packages & libraries

In [1]:
!apt-get install libgmp-dev libmpfr-dev libmpc-dev
!pip install gmpy2
!pip install gensafeprime
!pip install ecdsa[gmpy2]
!pip install "pycryptodome==3.9.7"
!pip install base58
!pip install ecdsa
!pip install utilitybelt
!pip install bitcoinaddress
!pip install bit
!pip install pyopenssl
!pip install cryptography

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgmp-dev is already the newest version (2:6.1.2+dfsg-2).
libmpc-dev is already the newest version (1.1.0-1).
libmpfr-dev is already the newest version (4.0.1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
import random as rnd
import random
import math
import time
import sys

import hashlib 
import gmpy2
from Crypto.Hash import RIPEMD160
from ecdsa import SigningKey, SECP256k1

import base58
import ecdsa 
import codecs

from ecdsa.keys import SigningKey
from utilitybelt import dev_random_entropy
from binascii import hexlify, unhexlify

import secrets
import hashlib as h

from bitcoinaddress import Wallet
from bit import PrivateKeyTestnet

### Exercise 10

(a)

## **1st Implementation**

In [3]:
def leading_zeros(num):
  n = 0
  for i in num:
    if i == '0': 
      n = n + 1
    else: 
      break

  return int(n/2)

The following function is used to generate keys.

In [4]:
def keyGen():
  private_key = SigningKey.generate(curve=SECP256k1)
  public_key = private_key.verifying_key
  
  return (private_key.to_string(), b'\04'+public_key.to_string())

The following function is used to create the base58 converter.

In [5]:
def base58convert(address):
  code_string = "123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz"
  x = int(address, 16)
  output = ""

  while x > 0:
    (x, r) = gmpy2.f_divmod(x, 58)
    output = output + code_string[r]

  n = leading_zeros(address)
  for i in range(n):
    output = output + code_string[0]

  return output[::-1]

The following function is used to check if Base58.

payload: PubKey hash in hex string form?

In [6]:
def Base58Check(payload, version='0x00'):
  address = version[2:]+payload
  b_address = bytes.fromhex(address)
  h = hashlib.sha256(b_address)
  h = hashlib.sha256(h.digest())

  # take the first 4 bytes
  checksum = str(h.hexdigest()[:8])

  # 25-byte binary bitcoin address
  address = address+checksum
  address = base58convert(address)

  return address

The following function is used to calculate the Hash160.

public_key: in bytes form

In [7]:
def getHash160(public_key):
  public_key_hash = hashlib.sha256(public_key)
  public_key_hash = RIPEMD160.new(public_key_hash.digest())

  return public_key_hash

The following function is used to get the address.

* public_key: the decimal number as a string
* version: choose a prefix and create a different address https://en.bitcoin.it/wiki/List_of_address_prefixes

In [8]:
def getAddress(public_key, version='0x00'):
  if isinstance(public_key, bytes):
    public_key = public_key
  else:
    public_key = public_key.encode()
  
  public_key_hash = getHash160(public_key)
  address = Base58Check(public_key_hash.hexdigest(), version)

  return address

Key Generation

In [9]:
(private_key, public_key) = keyGen()

print("Private Key:\n", private_key)
print()
print("Public Key:\n", public_key)
print()

Private Key:
 b'\xf4\x8b\xe1\xb1R\xed\xff\x19\xe5:\x02\\\x013E\xe9\xa1\xe6\x9a\x0eD\xf7\x82\x86\xa1\x14\x95\xf7\xb6\x04_\xeb'

Public Key:
 b'\x04~%\xc4S)\x96\xde\xcc\xc0\xae8\n\xab\xa5\x85\x0b\x9b\xac\x99\xf31k\xf5\x8a\xd3\x0c\xbd{\xa9\x9f;h\x7f\xae5\xbf\xfc$H\xa3\xa9\xcd\x1c>\xba\xdf>.NfH/\x1d\xdd\xa78]\x0bX\xa0\xf5\xb3\xe4\x1c'



In [10]:
# copy-paste the private key that was printed
private_key =   b'\xf4\x8b\xe1\xb1R\xed\xff\x19\xe5:\x02\\\x013E\xe9\xa1\xe6\x9a\x0eD\xf7\x82\x86\xa1\x14\x95\xf7\xb6\x04_\xeb'
pk = SigningKey.from_string(private_key, curve=SECP256k1)
public_key = SigningKey.from_string(private_key, curve=SECP256k1).verifying_key.to_string()

In [11]:
print("Private key in hex:\n", private_key.hex())
print()
print("Public key in hex:\n", public_key.hex())
print()

Private key in hex:
 f48be1b152edff19e53a025c013345e9a1e69a0e44f78286a11495f7b6045feb

Public key in hex:
 7e25c4532996deccc0ae380aaba5850b9bac99f3316bf58ad30cbd7ba99f3b687fae35bffc2448a3a9cd1c3ebadf3e2e4e66482f1ddda7385d0b58a0f5b3e41c



In [12]:
testnet_private_key = Base58Check(private_key.hex(), '0xef')
print("testnet Private key:\n", testnet_private_key)
print()

testnet Private key:
 93ScmnwypoRR1YXkwkSDaQrkCYA5NGNTYnTCCAY7Q9e9uMnjRcR



Address #1

In [13]:
address = getAddress(public_key, '0x6f')
print(address) 

mjBJH4h9xcuSayENoYCvVjcjSBFMJSiGpM


Check code - given address

In [14]:
AM = '03112345'
print(getAddress(AM, '0x6F'))

mkaWYS2DeChGv3u5tZMR59WSQkJEk61E3k


Address #2

In [15]:
AM = '03117176'
print(getAddress(AM, '0x6F'))

mtUSi5325xxiSH2am1iXtbZs9CpPsA2o3n


## **2nd Implementation**

The following cell is used to calculate the ripe hash.

In [16]:
def ripe_hash(key):
    ret = hashlib.new('ripemd160')
    ret.update(hashlib.sha256(key).digest())
    
    return ret.digest()

The following cell is used to calculate the double hash.

In [17]:
def double_hash(key):
    key=unhexlify(key)
    
    return hashlib.sha256(hashlib.sha256(key).digest()).digest()

The following cell is used to calculate a random secret exponent

In [18]:
def random_secret_exponent(curve_order):
    while True:
        random_hex = hexlify(dev_random_entropy(32))
        random_int = int(random_hex, 16)
        if random_int >= 1 and random_int < curve_order:
            
            return random_int

The following cell is used to generate the address.

In [19]:
def generate_address(public_key,isstr):
    public_key_bytes = codecs.decode(public_key, 'hex')
    
    if not isstr:
        public_key_bytes=hexlify(public_key_bytes)
    
    sha256_bpk = hashlib.sha256(public_key_bytes)
    sha256_bpk_digest = sha256_bpk.digest()
    
    ripemd160_bpk = hashlib.new('ripemd160')
    ripemd160_bpk.update(sha256_bpk_digest)
    ripemd160_bpk_digest = ripemd160_bpk.digest()
    ripemd160_bpk_hex = codecs.encode(ripemd160_bpk_digest, 'hex')

    public_key_and_version_hex = b'6f' + ripemd160_bpk_hex 
    
    checksum = hexlify(double_hash(public_key_and_version_hex)[:4])

    return base58.b58encode(unhexlify(public_key_and_version_hex + checksum))

The following cell is used to generate the private key.

In [20]:
def generate_private_key():
    curve = ecdsa.curves.SECP256k1
    se = random_secret_exponent(curve.order)
    from_secret_exponent = ecdsa.keys.SigningKey.from_secret_exponent
    
    return hexlify(from_secret_exponent(se, curve, hashlib.sha256).to_string())

The following cell is used to generate the public key.

In [21]:
def generate_public_key(private_key_hex):
    
    secret = unhexlify(private_key_hex)
    
    order = ecdsa.SigningKey.from_string(secret, curve=ecdsa.SECP256k1).curve.generator.order()
    
    p = ecdsa.SigningKey.from_string(secret, curve=ecdsa.SECP256k1).verifying_key.pubkey.point
    
    x_str = ecdsa.util.number_to_string(p.x(), order)
    y_str = ecdsa.util.number_to_string(p.y(), order)
    
    if int(hexlify(y_str),16)%2==0:
        st="02"
    else:
        st="03"
        
    return st+str(hexlify(x_str))[2:-1]

In [22]:
private_key_hex = generate_private_key()

In [23]:
print("Private key:\n", private_key_hex)

Private key:
 b'7eaec39311c688b9570a749d6fc045c92a8861245c823bf9063b407ce2d6d286'


In [24]:
print("Private key in base58:\n", base58.b58encode(unhexlify(private_key_hex)))

Private key in base58:
 b'9XWw4RecRwv352JiSpMLi8WafHnfBZvSpL6fQAio1fkq'


In [25]:
public_adr_hex_1 = generate_address(generate_public_key(private_key_hex),True)

In [26]:
print("Public key:\n",public_adr_hex_1)

Public key:
 b'n3Qi1TTa4wZGWWVHnn4ugmxUpVjeLMXv5A'


In [27]:
public_adr_hex_2 = generate_address('03117176',False)

In [28]:
print("Public Address:\n",public_adr_hex_2)

Public Address:
 b'mtUSi5325xxiSH2am1iXtbZs9CpPsA2o3n'


(b)

*based on 1st Implementation*

In [29]:
S = 2**12
N = 2**S

The following function is used to count zeros at the begining.

In [30]:
def is_valid(transaction_id):
  cnt = 0
  for i in transaction_id:
    if i == '0': 
      cnt = cnt + 1
    else: 
      break

  return cnt >= 8

The following functions is used to calculate the nonce.

* produce a random nonce 
* number of bytes depends on N

In [31]:
def get_nonce():
  nonce = random.randint(0, N) 
  mylen = max(len(bin(nonce))//8+1, S//8)
  
  return nonce.to_bytes(mylen, 'big')

The following function is used for mining.

* transaction in hex string form without "0x" at the beginning

In [32]:
def mining(transaction):
  while True:
    nonce = get_nonce() # nonce in bytes form
    input = nonce+bytes.fromhex(transaction)
    
    transaction_id = hashlib.sha256(input)
    transaction_id = hashlib.sha256(transaction_id.digest())
    transaction_id = transaction_id.digest()[::-1] # reverse order

    if is_valid(transaction_id.hex()): 
      print("Found it!")
      break

  return transaction_id.hex()

The following cell takes time to run. See comments in report.

In [33]:
# transcript = '0100000002ef7e58660dd495edb75028989cc7eaf36f5c68b4bfe81568b11da5f10421ec45000000008a473044022027bd56aec670822205ab08e626842902e228da6482b6c94b5302011881c180850220727812d89ce21057839a06da87af05c75dffc7a5b4c576d46c2d8f7475be8e2c014104fab3f5793d3a9fb69cca99cce54650afcc8c3f59f09bf0812253f413f1690095427a19d1cf8e710b0f3aa0a928050af233c49afddf0e1496ff56ecf8144e2e22ffffffffc6dc2887a710a1dda56315dfec80921cfc83254952934c093553f17340b02225000000008b483045022100f28c66f5006e6b63283583831f0bd4af229bcd22bcc53beefc583d0442a0a511022056e893a987ddc2eec3af92b74af740269d9cbf8211807f2d28f74361d5049701014104fab3f5793d3a9fb69cca99cce54650afcc8c3f59f09bf0812253f413f1690095427a19d1cf8e710b0f3aa0a928050af233c49afddf0e1496ff56ecf8144e2e22ffffffff0340420f00000000001976a914f0e7b20a3e49e76c81fcd7d1b2c78ba78602b07188ac40420f00000000001976a914433ff80d52b9ceeb2ec0b5a29171f27a4c59c28d88ac80380100000000001976a9143752d558525339337ae23acfeda99fe1572b3bc288ac00000000'
# transcript_id = mining(transcript)
# print(transcript_id)